In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Turn off GPU
if False:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from tensorflow import keras
import tensorflow as tf
from glob import glob
import numpy as np

gpus = tf.config.experimental.list_physical_devices('GPU')

print(tf.__version__)
print(gpus)

if len(gpus) > 0:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024 * 1))])

2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from glob import glob

last_models = sorted(glob("model-saved/deeplabV3+_mobileNetV2/NEW_3/LAST_MODEL/*/*/*/*/*/*/*.ckpt"))
print(last_models, len(last_models))
modelPath = last_models[0]

['model-saved/deeplabV3+_mobileNetV2/NEW_3/LAST_MODEL\\True\\layer_12\\alpha_0.5\\loss_dice_coef\\l1_1e-06\\l2_0.0001\\VERTICAL_FLIP_False.ckpt', 'model-saved/deeplabV3+_mobileNetV2/NEW_3/LAST_MODEL\\True\\layer_12\\alpha_0.5\\loss_dice_coef\\l1_1e-06\\l2_0.0001\\VERTICAL_FLIP_True.ckpt'] 2


In [6]:
model = None

In [7]:
import models
from utils.model_utils import BatchGenerator
from utils.model_utils import toTFlite
from tensorflow import lite as tflite

last_models = [last_models[-1]]

for i, modelPath in enumerate(last_models):
    print(f"{i+1} of {len(last_models)}")
    modelSplit = modelPath.replace("/", "\\").split("\\")

    TEST =        modelSplit[4] == "True"
    deeplayer =   modelSplit[5].replace("layer_", "")
    alpha = float(modelSplit[6].replace("alpha_", ""))
    loss =        modelSplit[7]

    mobileLayers = {
        "shallowLayer": "block_2_project_BN",
        "deepLayer": f"block_{deeplayer}_project_BN"}
    
    model = models.deeplabV3(
        imageSize=(240, 320), nClasses=2, alpha=alpha, withArgmax=True, mobileLayers=mobileLayers, TEST=TEST)
    model.load_weights(modelPath)

    images = glob("utils\\split_3\\TrainVal\\img\\*")
    print(len(images))

    toTFlite(
        model=model,
        savePath=modelPath.replace(".ckpt", ""),
        representativeDatasetGen=BatchGenerator(images),
        supportedOps=[
            tflite.OpsSet.SELECT_TF_OPS,
            tflite.OpsSet.TFLITE_BUILTINS])
    model = None

1 of 1


d:\Anaconda\envs\tf-gpu\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


2975


INFO:tensorflow:Assets written to: C:\Users\rafae\AppData\Local\Temp\tmp5wehd0nr\assets


INFO:tensorflow:Assets written to: C:\Users\rafae\AppData\Local\Temp\tmp5wehd0nr\assets
d:\Anaconda\envs\tf-gpu\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
